In [2]:
import gzip, pickle
from torch import tensor

In [3]:
from torch.functional import F

In [14]:
from torch import nn
import torch

In [4]:
def get_data():
    # path = datasets.download_data(MNIST_URL, ext='.gz')
    path = '/content/mnist.pkl.gz'
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

In [5]:
x_train, y_train, x_valid, y_valid = get_data()
nh, bs = 50, 512
c = y_train.max().item()+1
loss_func = F.cross_entropy

In [16]:
class DummyModule():
    def __init__(self, n_in, nh, n_out):
        self._modules = {}
        self.l1 = nn.Linear(n_in,nh)
        self.l2 = nn.Linear(nh,n_out)
        
    def __setattr__(self,k,v):
        if not k.startswith("_"): self._modules[k] = v
        super().__setattr__(k,v)
        
    def __repr__(self): return f'{self._modules}'
    
    def parameters(self):
        for l in self._modules.values():
            for p in l.parameters(): yield p

    def __call__(self, x): return self.l2(F.relu(self.l1(x)))

    def zero_grad(self):
        for p in self.parameters():
            p.grad.data.zero_()

In [17]:
def fit():
    for epoch in range(epochs):
        for i in range((n-1)//bs + 1):
            start_i = i*bs
            end_i = start_i+bs
            xb = x_train[start_i:end_i]
            yb = y_train[start_i:end_i]
            loss = loss_func(model(xb), yb)

            loss.backward()
            with torch.no_grad():
                for p in model.parameters(): p -= p.grad * lr
            model.zero_grad()

In [18]:
epochs, n, lr = 1, x_train.shape[0], 0.5
model = DummyModule(x_train.shape[1], nh, 10)
fit()

---

In [82]:
#  A8
class Optimizer():
    def __init__(self, params, lr): self.params, self.lr = list(params), lr
    def zero_grad(self):
        for p in self.params:
            p.grad.data.zero_()

    def step(self):
        with torch.no_grad():
            for p in self.params:
                p -= p.grad * self.lr

def fit():
    for epoch in range(epochs):
        for i in range((n-1)//bs + 1):
            start_i = i*bs
            end_i = start_i+bs
            xb = x_train[start_i:end_i]
            yb = y_train[start_i:end_i]
            loss = loss_func(model(xb), yb)

            loss.backward()
            opt.step()
            opt.zero_grad()

In [83]:
model = nn.Sequential(nn.Linear(x_train.shape[1], nh), nn.ReLU(), nn.Linear(nh, c))
# accuracy-before train
print((model(x_train[:bs]).max(-1).indices == y_train[:bs]).sum()/ bs)
opt = Optimizer(model.parameters(), 0.9)
fit()
print((model(x_train[:bs]).max(-1).indices == y_train[:bs]).sum()/ bs)

tensor(0.1250)
tensor(0.9062)


In [74]:
from torch import optim
def get_model(model_func, lr=0.9):
    model = nn.Sequential(*model_func())
    return model, optim.SGD(model.parameters(), lr=lr)

In [75]:
def get_layers():
    return nn.Linear(x_train.shape[1], nh), nn.ReLU(), nn.Linear(nh, c)

In [86]:
epochs = 100
model, opt = get_model(get_layers, lr = 0.001)
print((model(x_train[:bs]).max(-1).indices == y_train[:bs]).sum()/ bs)
fit()
print((model(x_train[:bs]).max(-1).indices == y_train[:bs]).sum()/ bs)

tensor(0.0586)
tensor(0.8574)


----